In [1]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go




In [2]:
dfs, available_classes, available_models = read_raw_reports(os.path.join('..',raw_reports_path))

In [3]:
available_classes_r = available_classes[::-1]

In [4]:
available_classes_r

['sett',
 'paving stones',
 'ground',
 'grass',
 'concrete plates',
 'concrete',
 'asphalt']

In [5]:
available_models

['ViT-L_14',
 'EVA02-E-14-plus',
 'ViT-H-14-378-quickgelu',
 'ViT-B_32',
 'ViT-bigG-14-CLIPA-336',
 'RN50x64',
 'ViT-SO400M-14-SigLIP-384',
 'RN101']

In [6]:
available_classes

['asphalt',
 'concrete',
 'concrete plates',
 'grass',
 'ground',
 'paving stones',
 'sett']

In [7]:
all_dfs = []
modelwise_dfs = {}

for class_dict in dfs:
    for model_name in dfs[class_dict]:
        model_df = dfs[class_dict][model_name]
        all_dfs.append(model_df)

        if not model_name in modelwise_dfs:
            modelwise_dfs[model_name] = []

        modelwise_dfs[model_name].append(model_df)


for model_name in modelwise_dfs:
    modelwise_dfs[model_name] = pd.concat(modelwise_dfs[model_name]).copy()

In [8]:
merged_df = pd.concat(all_dfs)

In [9]:
true_classes = merged_df.groupby('image_name').agg('first')[class_label].str.replace('_', ' ')
merged_pred = merged_df.groupby('image_name').sum(numeric_only=True)

merged_pred[prediction_label] = merged_pred.idxmax(axis=1)
merged_pred[class_label] = true_classes


In [10]:
merged_pred

,asphalt,concrete,concrete plates,grass,ground,paving stones,sett,prediction,class
image_name,,,,,,,,,
00b7d076504ff9e7b7ed395b1ad8da2f.png,0.480256,0.670878,0.458242,0.857903,4.166124,0.867546,0.499050,ground,ground
013c4d1eb8f0e6d09d3506e69c2677c4.png,0.258615,0.254482,0.076824,0.145211,0.347741,5.989711,0.927621,paving stones,paving stones
0144ebc3606a3de74528b5a0353f8df0.png,0.350016,0.571236,0.223059,0.146684,0.276353,5.372983,1.059888,paving stones,paving stones
01587414f7e8693a05f13f9ba4457798.png,1.008546,0.506683,1.324714,0.168768,0.546040,3.787260,0.657536,paving stones,paving stones
01beabd884a7d4d05af069ec25c096bf.png,0.207833,0.173900,0.211628,0.171041,0.424571,5.749014,1.061423,paving stones,paving stones
...,...,...,...,...,...,...,...,...,...
fef13573ee306eb702444d42b069ea91.png,0.301393,0.144404,0.204715,5.079870,1.436470,0.478786,0.354594,grass,grass
fefe0b8ce34118a31559f138cb1f1c05.png,0.412514,0.165985,0.112019,0.121229,0.228178,5.785807,1.174724,paving stones,sett
fefe69a9ccf153a319a2efa038885ce5.png,0.218333,0.263272,0.269892,0.135827,0.174470,6.424686,0.513515,paving stones,paving stones


In [11]:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r, normalize='true') * 100
c_mat = c_mat.round(3)

In [12]:
# sanity check on labels:
print(merged_pred[class_label].unique())

['ground' 'paving stones' 'asphalt' 'sett' 'grass' 'concrete plates'
 'concrete']


In [13]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [14]:
n_rows = int(ceil(len(available_models)/2))

In [15]:
modelwise_confusion_matrices = {}

for model in modelwise_dfs:
    true_labels =      modelwise_dfs[model][class_label]#.tolist()
    predicted_labels = modelwise_dfs[model][prediction_label]#.tolist()

    modelwise_confusion_matrices[model] = confusion_matrix(true_labels,predicted_labels,labels=available_classes, normalize='true') * 100
    modelwise_confusion_matrices[model] = modelwise_confusion_matrices[model].round(3)

In [16]:
modelwise_confusion_matrices

{'EVA02-E-14-plus': array([[69.231, 30.769,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   , 25.   ,  0.   ,  0.   , 75.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 3.077,  1.538,  0.   , 95.385,  0.   ,  0.   ,  0.   ],
        [ 1.802,  0.   ,  0.   ,  4.505, 91.892,  0.   ,  1.802],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 1.786,  0.   ,  0.   ,  0.   ,  1.786, 96.429,  0.   ]]),
 'RN101': array([[23.077,  0.   , 10.256,  5.128, 33.333, 15.385, 12.821],
        [25.   ,  0.   , 25.   , 25.   ,  0.   , 25.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   , 84.615, 12.308,  0.   ,  3.077],
        [ 0.   ,  0.   ,  6.306,  0.   , 63.063, 29.73 ,  0.901],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 1.19 ,  0.   ,  2.976,  0.   ,  0.   , 95.833,  0.   ]]),
 'RN50x64': array([[ 5.128, 46.154,  0.   , 

In [17]:
# fig = go.Figure().set_subplots(rows=n_rows, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.05, horizontal_spacing=0.01)
fig = make_subplots(rows=n_rows, cols=2, 
                    shared_xaxes=True, 
                    shared_yaxes=True,
                    subplot_titles=list(modelwise_confusion_matrices.keys()),
                    vertical_spacing=0.015, 
                    horizontal_spacing=0.01,
                    # x_title="Prediction",
                    # y_title="Class",
                    )

for i,matrix in enumerate(modelwise_confusion_matrices):
    subplot = px.imshow(modelwise_confusion_matrices[matrix].T,
                
                x=available_classes,
                y=available_classes,
              text_auto=True, 
              #  aspect="auto",
               )
    
    # print(i//2+1,i%2+1)

    fig.add_trace(subplot.data[0], row=i//2+1, col=i%2+1)
    # fig.append_trace(subplot.data[0], row=i//2+1, col=i%2+1)

fig.update_coloraxes(colorscale=px.colors.sequential.Burg)
fig.update_layout(height=400*n_rows, width=400*2,
                  xaxis_title="Prediction",
                  yaxis_title="Class",
                  showlegend=True,)
fig.show()


In [18]:
# getting the diagonal sum of the confusion matrix:
diagonals = {}

for model in modelwise_confusion_matrices:
    diagonals[model] = modelwise_confusion_matrices[model].diagonal().sum()

diagonals['combined'] = c_mat.diagonal().sum() 

diagonals_df = pd.DataFrame.from_dict(diagonals, orient='index',columns=['score']).reset_index().rename(columns={'index':'model_name'}).sort_values(by='score', ascending=False)

diagonals_df['score'] = round(100 *  diagonals_df['score'] / (len(available_models) * 100),2)

diagonals_df

,model_name,score
8,combined,46.94
5,ViT-H-14-378-quickgelu,40.90
7,ViT-SO400M-14-SigLIP-384,38.16
0,EVA02-E-14-plus,35.19
6,ViT-L_14,26.71
4,ViT-B_32,24.67
1,RN101,21.34
2,RN50x64,19.97
3,ViT-bigG-14-CLIPA-336,12.28


In [19]:
fig = px.bar(diagonals_df,x='model_name', y='score', text_auto=True)
fig.show()


In [20]:
selected_models = {}

for modelname in ['ViT-H-14-378-quickgelu', 'ViT-SO400M-14-SigLIP-384', 'EVA02-E-14-plus']:
    selected_models[modelname] = modelwise_dfs[modelname]

In [21]:
merged_selected = pd.concat(selected_models).reset_index()

In [22]:
true_classes = merged_selected.groupby('image_name').agg('first')[class_label].str.replace('_', ' ') #.reset_index().rename(columns={class_label:'class'})

In [23]:
merged_selected = merged_selected.groupby('image_name').sum(numeric_only=True)

In [24]:
merged_selected.drop(columns=['level_1'], inplace=True)

In [25]:
merged_pred[prediction_label] = merged_selected.idxmax(axis=1)
merged_pred[class_label] = true_classes.tolist()

In [26]:
# computing the confusion matrix:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r, normalize='true') * 100
c_mat = c_mat.round(3)

In [27]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [28]:
100 * (c_mat.diagonal().sum() / (100 * len(available_models)))

50.99287499999999